In [55]:
#1
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [56]:
#2
df = pd.read_csv('/Users/luryan/Documents/persona_project/spaceship-titanic/data/train.csv')

# Feature engineering
df['Homeplanet_Mars'] = df['HomePlanet'].apply(lambda x: 1 if x == 'Mars' else 0)
df['Homeplanet_Earth'] = df['HomePlanet'].apply(lambda x: 1 if x == 'Earth' else 0)
df['Homeplanet_Europa'] = df['HomePlanet'].apply(lambda x: 1 if x == 'Europa' else 0)

if 'HomePlanet' in df.columns:
    df = df.drop('HomePlanet', axis=1)

df['Cabin_S'] = df['Cabin'].apply(lambda x: 1 if str(x).split('/')[-1] == 'S' else 0)
df['Cabin_P'] = df['Cabin'].apply(lambda x: 1 if str(x).split('/')[-1] == 'P' else 0)

if 'Cabin' in df.columns:
    df = df.drop('Cabin', axis=1)

drop = ['PassengerId', 'Destination', 'Name']
df = df.drop(drop, axis=1)

df = df.fillna(df.mean())

coluna_alvo = 'Transported'

X = df.drop(coluna_alvo, axis=1)
y = df[coluna_alvo]

# Padronizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [57]:
#3
modelo = Sequential()
modelo.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
modelo.add(Dropout(0.3))
modelo.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.2), input_dim=100))
modelo.add(Dense(64, activation='relu'))
modelo.add(Dense(1, activation='sigmoid'))

modelo.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['accuracy'])

# Treinar o modelo
modelo.fit(X_train, y_train, epochs=10, batch_size=22, validation_data=(X_test, y_test))

# Fazer previsões
predictions = modelo.predict(X_test)

# Avaliar a acurácia do modelo
_, acuracia = modelo.evaluate(X_test, y_test)
print(f'Acurácia do modelo: {acuracia}')

print(f'Length of index: {len(df.index)}')
print(f'Length of predictions: {len(predictions)}')

Epoch 1/10
313/313 [==============================] - 7s 8ms/step - loss: 2.0571 - accuracy: 0.7472 - val_loss: 0.1674 - val_accuracy: 0.8087
Epoch 2/10
313/313 [==============================] - 2s 7ms/step - loss: 0.1642 - accuracy: 0.7792 - val_loss: 0.1549 - val_accuracy: 0.7907
Epoch 3/10
313/313 [==============================] - 3s 11ms/step - loss: 0.1566 - accuracy: 0.7865 - val_loss: 0.1429 - val_accuracy: 0.8134
Epoch 4/10
313/313 [==============================] - 3s 11ms/step - loss: 0.1546 - accuracy: 0.7901 - val_loss: 0.1441 - val_accuracy: 0.8081
Epoch 5/10
313/313 [==============================] - 3s 10ms/step - loss: 0.1528 - accuracy: 0.7904 - val_loss: 0.1433 - val_accuracy: 0.8064
Epoch 6/10
313/313 [==============================] - 4s 14ms/step - loss: 0.1521 - accuracy: 0.7899 - val_loss: 0.1419 - val_accuracy: 0.8122
Epoch 7/10
313/313 [==============================] - 4s 14ms/step - loss: 0.1493 - accuracy: 0.7923 - val_loss: 0.1453 - val_accuracy: 0.8064
E

In [58]:
#4
X_test_df = pd.DataFrame(X_test, columns=df.columns[:-1])  # Assuming the last column is the target variable
final = pd.DataFrame({'PassengerId': df.loc[X_test_df.index].index, 'Transported': predictions.flatten() > 0.5})
# final.to_csv('/Users/luryan/Documents/persona_project/spaceship-titanic/output_submission.csv', index=False)

In [59]:
#5
test_df_submission = pd.read_csv('/Users/luryan/Documents/persona_project/spaceship-titanic/data/test.csv')

test_df_submission['Homeplanet_Mars'] = test_df_submission['HomePlanet'].apply(lambda x: 1 if x == 'Mars' else 0)
test_df_submission['Homeplanet_Earth'] = test_df_submission['HomePlanet'].apply(lambda x: 1 if x == 'Earth' else 0)
test_df_submission['Homeplanet_Europa'] = test_df_submission['HomePlanet'].apply(lambda x: 1 if x == 'Europa' else 0)

if 'HomePlanet' in test_df_submission.columns:
    test_df_submission = test_df_submission.drop('HomePlanet', axis=1)

test_df_submission['Cabin_S'] = test_df_submission['Cabin'].apply(lambda x: 1 if str(x).split('/')[-1] == 'S' else 0)
test_df_submission['Cabin_P'] = test_df_submission['Cabin'].apply(lambda x: 1 if str(x).split('/')[-1] == 'P' else 0)

if 'Cabin' in test_df_submission.columns:
    test_df_submission = test_df_submission.drop('Cabin', axis=1)

# Preencher valores ausentes em colunas específicas
columns_to_fillna = ['Homeplanet_Mars', 'Homeplanet_Earth', 'Homeplanet_Europa', 'Cabin_S', 'Cabin_P']
test_df_submission[columns_to_fillna] = test_df_submission[columns_to_fillna].fillna(0)  # Preencher com 0, você pode ajustar conforme necessário

#6
X_submission = scaler.transform(test_df_submission.drop(['PassengerId', 'Destination', 'Name'], axis=1))
predictions_submission = modelo.predict(X_submission)
n_predictions_submission = (predictions_submission > 0.5).astype(bool)

# Criar DataFrame para submissão
output_submission = pd.DataFrame({'PassengerId': test_df_submission['PassengerId'], 'Transported': n_predictions_submission.squeeze()})

# Salvar DataFrame de submissão em arquivo CSV
output_submission.to_csv('/Users/luryan/Documents/persona_project/spaceship-titanic/output_submission.csv', index=False)


134/134 [==============================] - 1s 4ms/step


In [60]:
#6
predictions = modelo.predict(X_test)

_, acuracia = modelo.evaluate(X_test, y_test)
print(f'Acurácia do modelo: {acuracia}')

print(f'Length of index: {len(df.index)}')
print(f'Length of predictions: {len(predictions)}')

54/54 [==============================] - 0s 5ms/step - loss: 0.1399 - accuracy: 0.8105
Acurácia do modelo: 0.8104650974273682
Length of index: 8599
Length of predictions: 1720
